In [ ]:
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import numpy as np
import math
import warnings
import time
import scipy.stats as st
import pandas as pd
import sys
import scipy.optimize as op
warnings.filterwarnings("ignore")

def bee_eq(y, t, w1,w2, alpha, external_temperature, peri_, treatment,h):
    hive_temperature = y
    #dydt = -w1*(1/(1+np.exp(-hive_temperature))) + w2 + 2*(-hive_temperature + external_temperature)
    if treatment == 0:
        dydt = -h*(w1*w2*(1-np.exp(-alpha*hive_temperature)) / (w2+w1*np.exp(-alpha*hive_temperature)))+ 2*(-hive_temperature + external_temperature) #- np.sum(hive_temperature)
    else:
        dydt = -h*(w1*w2*(1-np.exp(-alpha*hive_temperature)) / (w2+w1*np.exp(-alpha*hive_temperature))) + 2*(-hive_temperature) + peri_ + external_temperature #- np.sum(hive_temperature)
    return dydt

def run_bee_eq(t, a, w1, w2, alpha, ext_temp, peri_, treatment,h):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, alpha, ext_temp, peri_, treatment,h), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data,l):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    w1 = 80
    w2 = 221
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value
        theta_ideal = ps['theta_'+str(k)].value
        ext_temp_ = n['Temp_Box'] - theta_ideal
        truth_ = n['Temp_'+str(hive_no)] - theta_ideal
        peri_ = n['Temp_Easy_'+str(hive_no)] - theta_ideal

        try:
            treatment = n['Treatment'].values[0]
        except:
            treatment = 0
        t_max = len(n['Temp_Box'])-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(ext_temp_)
        fitted = run_bee_eq(t, a, w1, w2, alpha, ext_temp_, peri_, treatment, h)
        model = np.concatenate((model, fitted + theta_ideal))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data['Temp_'+str(hive_no)]).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    print(np.mean(abs((model - data['Temp_'+str(hive_no)]).ravel())))
    return return_value

import pandas as pd
hive_no = 294
data = pd.read_csv('/home/shamima/lm_optimization/C_294-no_hiccups.csv')

h_max = 1
lambda_ = 96/h_max
w1_max = np.inf
w2_max = np.inf
print("w1 = " + str(w1_max) +", w2 = " +str(w2_max)+ " , "+ str(hive_no)+" , lambda = " +str(lambda_)+", h = ," + str(h_max))
d = pd.DataFrame(data).groupby([data['Date']])

l = lambda_
params = Parameters()
for i in range(len(d)):
    params.add('h_'+str(i), value = np.random.ranf(), min = 0.1, max = h_max)

for i in range(len(d)):
    params.add('theta_'+str(i), value = 35, min = 31, max = 38)    
params.add('w1', value= 30, min=0, max=w1_max)
params.add('w2', value= 40, min=0, max=w2_max)

t_max = len(data['Temp_Box'])-1
t = np.linspace(0,t_max, num = t_max+1)
result = minimize(residual, params, args=(t,data,l),method='least_squares',nan_policy='omit')
sig_min = data['Temp_'+str(hive_no)] + result.residual[:len(data['Temp_'+str(hive_no)])].reshape(data['Temp_'+str(hive_no)].shape)
param_min = []
for i in params:
    param_min.append(result.params[i].value)
error = math.sqrt(np.mean((result.residual[:len(data['Temp_'+str(hive_no)])].reshape(data['Temp_'+str(hive_no)].shape)**2)))

print('sig')
for i in sig_min:
    print(i)

print('parameters')
for i in param_min:
    print(i)

print('error', error)
print('loss', math.sqrt(np.mean((result.residual)**2)))